In [1]:
import tenseal.sealapi as seal
import util

In SEAL, 'parms_id' is the unique 256-bit hash identifier of the encryption paramters. When any of the parameters change, `parms_id` will change. 

When a SEALContext is created from a given EncryptionParameters instance, Microsoft SEAL automatically creates a so-called `modulus switching chain`, which is a chain of other encryption parameters derived from the original set. The parameters in the modulus switching chain are the same as the original parameters with the exception that size of the coefficient modulus is decreasing going down the chain. More precisely, each parameter set in the chain attempts to remove the last coefficient modulus prime from the previous set; this continues until the parameter set is no longer valid
(e.g., `plain_modulus` is larger than the remaining `coeff_modulus`). It is easy to walk through the chain and access all the parameter sets. Additionally,
each parameter set in the chain has a `chain index' that indicates its position in the chain so that the last set has index 0. We say that a set
of encryption parameters, or an object carrying those encryption parameters, is at a higher level in the chain than another set of parameters if its the
chain index is bigger, i.e., it is earlier in the chain.

Each set of parameters in the chain involves unique pre-computations performed when the SEALContext is created, and stored in a `SEALContext::ContextData`
object. The chain is basically a linked list of `SEALContext::ContextData` objects, and can easily be accessed through the SEALContext at any time. Each
node can be identified by the parms_id of its specific encryption parameters (poly_modulus_degree remains the same but coeff_modulus varies).


In [2]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.BFV)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)

In this example we use a custom coeff_modulus, consisting of 5 primes of sizes 50, 30, 30, 50, and 50 bits. Note that this is still OK according to the explanation in `1_bfv_basics.cpp`. Indeed, `CoeffModulus::MaxBitCount(poly_modulus_degree)` returns 218 (greater than 50+30+30+50+50=210).
Due to the modulus switching chain, the order of the 5 primes is significant. The last prime has a special meaning and we call it the `special prime'. Thus, the first parameter set in the modulus switching chain is the only one that involves the special prime. All key objects, such as SecretKey, are created at this highest level. All data objects, such as Ciphertext, can be only at lower levels. The special prime should be as large as the largest of the other primes in the coeff_modulus, although this is not a strict requirement.

In [8]:
print("Max bit count: " + str(seal.CoeffModulus.MaxBitCount(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128)))

parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [50, 30, 30, 50, 50]))
parms.set_plain_modulus(seal.PlainModulus.Batching(poly_modulus_degree, 20))

context = seal.SEALContext(parms, True, seal.SEC_LEVEL_TYPE.TC128)
util.print_parameters(context)

Max bit count: 218
/
| Encryption parameters: 
|    scheme: BFV
|    poly_modulus_degree: 8192
|    coeff_modulus_size: 210 (50 30 30 50 50) bits
|    plain_modulus: 1032193


There are convenience method for accessing the SEALContext::ContextData for some of the most important levels:

- SEALContext::key_context_data(): access to key level ContextData 
- SEALContext::first_context_data(): access to highest data level ContextData
- SEALContext::last_context_data(): access to lowest level ContextData

We iterate over the chain and print the parms_id for each set of parameters.

In [16]:
context_data = context.key_context_data()
print("---> Level (chain index): " + str(context_data.chain_index()), end=' ')
print(".... key_context_data()")

print("\tparms_id: " + str(context_data.parms_id()))

print("\tcoeff_modulus primes: ", end = '')
for prime in context_data.parms().coeff_modulus():
    print(prime.value(), end = ' ')
    

---> Level (chain index): 4 .... key_context_data()
	parms_id: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
	coeff_modulus primes: 1125899905744897 1073643521 1073692673 1125899906629633 1125899906826241 

Now, look at the remaining (data) levels.

In [17]:
context_data = context.first_context_data()

while context_data:
    print("Level (chain index): " + str(context_data.chain_index()), end='')
    if context_data.parms_id()==context.first_parms_id():
        print(".... first_context_data(")
    elif context_data.parms_id() == context.last_parms_id():
        print(".... last_context_data()")
    else:
        print("")
    
    print("\tparms_id: " + str(context_data.parms_id()))
    print("\tcoeff_modulus primes: ", end='')
    for prime in context_data.parms().coeff_modulus():
        print(prime.value(), end=' ')
    print("\\ ")
    print("\\-->")

    context_data = context_data.next_context_data()

print("End of chain reached.")

Level (chain index): 3.... first_context_data(
	parms_id: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]
	coeff_modulus primes: 1125899905744897 1073643521 1073692673 1125899906629633 \ 
\-->
Level (chain index): 2
	parms_id: [9611362035343820607, 16250750482064473934, 11666325943188447289, 11197906540159540193]
	coeff_modulus primes: 1125899905744897 1073643521 1073692673 \ 
\-->
Level (chain index): 1
	parms_id: [8338375294373729721, 6255090713888968186, 9221042086196212239, 13262945982911515329]
	coeff_modulus primes: 1125899905744897 1073643521 \ 
\-->
Level (chain index): 0.... last_context_data()
	parms_id: [12645946865612918007, 3410116064097512307, 265341692546732382, 11895432757488484810]
	coeff_modulus primes: 1125899905744897 \ 
\-->
End of chain reached.


Create some keys and check that they appear at the highest level.

In [19]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

print( "Print the parameter IDs of generated elements.")
print("\t+ public_key: " + str(public_key.parms_id()))
print("\t+ secret_key: " + str(secret_key.parms_id()))
print("\t+ relin_key: " + str(relin_keys.parms_id()))

Print the parameter IDs of generated elements.
	+ public_key: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
	+ secret_key: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
	+ relin_key: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]


Create `encryptor`, `evaluator`, and `decryptor`.

In [21]:
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

In [33]:
plain = seal.Plaintext("1x^3 + 2x^2 + 3x^1 + 4")
encrypted = seal.Ciphertext()
encryptor.encrypt(plain, encrypted)
print("\t+ plain: " + str(plain.parms_id()))
print("\t+ encrypted: " + str(encrypted.parms_id()))

	+ plain: [0, 0, 0, 0]
	+ encrypted: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]


`Modulus switching`is used to change ciphertext parameters down in the chain. `Evaluator::mod_switch_to_next` always switches to the next level down the chain, whereas `Evaluator::mod_switch_to` switches to a parameter set down the chain corresponding to a given `parms_id`. However, it is impossible to switch up in the chain.

In [34]:
print("Perform modulus switching on encrypted and print.")
context_data = context.first_context_data()
print("--->")
while context_data.next_context_dat]a():
    print("Level (chain index): " + str(context_data.chain_index()))
    print("\tparms_id of encrypted: " + str(encrypted.parms_id()))
    print("\tNoise budget at this level: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")
    print("\\")
    print("\\-->")
    evaluator.mod_switch_to_next_inplace(encrypted)
    context_data = context_data.next_context_data()

print("Level (chain index): " + str(context_data.chain_index()))
print("\tparms_id of encrypted: " + str(encrypted.parms_id()))
print("\tNoise budget at this level: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")
print("\\")
print("\\-->")
print("End of chain reached. ")

Perform modulus switching on encrypted and print.
--->
Level (chain index): 3
	parms_id of encrypted: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]
	Noise budget at this level: 132 bits
\
\-->
Level (chain index): 2
	parms_id of encrypted: [9611362035343820607, 16250750482064473934, 11666325943188447289, 11197906540159540193]
	Noise budget at this level: 82 bits
\
\-->
Level (chain index): 1
	parms_id of encrypted: [8338375294373729721, 6255090713888968186, 9221042086196212239, 13262945982911515329]
	Noise budget at this level: 52 bits
\
\-->
Level (chain index): 0
	parms_id of encrypted: [12645946865612918007, 3410116064097512307, 265341692546732382, 11895432757488484810]
	Noise budget at this level: 22 bits
\
\-->
End of chain reached. 


In [36]:
print("Decrypt still works after modulus switching.")
decryptor.decrypt(encrypted, plain)
print("\t+ Decryption of encrypted: " + plain.to_string() +  ".... Correct")

Decrypt still works after modulus switching.
	+ Decryption of encrypted: 1x^3 + 2x^2 + 3x^1 + 4.... Correct


In [37]:
print("Computation is more efficient with modulus switching.")
print("Compute the 8th power. ")
encryptor.encrypt(plain, encrypted)
print("\t+ Noise budget fresh: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)
print("\t+ Noise budget of 2nd power: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)
print("\t+ Noise budget of the 4th power: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")

evaluator.mod_switch_to_next_inplace(encrypted)
print("\t+ Noise budget after modulus switching: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")

Computation is more efficient with modulus switching.
Compute the 8th power. 
	+ Noise budget fresh: 132 bits
	+ Noise budget of 2nd power: 100 bits
	+ Noise budget of the 4th power: 67 bits
	+ Noise budget after modulus switching: 67 bits


In [38]:
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)
print("\t+ Noise budget of the 8th power: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.mod_switch_to_next_inplace(encrypted)
print("\t+Noise budget after modulus switching: " + str(decryptor.invariant_noise_budget(encrypted)) + " bits")
decryptor.decrypt(encrypted, plain)
print("\t+Decryption of the 8th power (hexadecimal) .... Correct.")
print("\t" + plain.to_string())

	+ Noise budget of the 8th power: 34 bits
	+Noise budget after modulus switching: 34 bits
	+Decryption of the 8th power (hexadecimal) .... Correct.
	1x^24 + 10x^23 + 88x^22 + 330x^21 + EFCx^20 + 3A30x^19 + C0B8x^18 + 22BB0x^17 + 58666x^16 + C88D0x^15 + 9C377x^14 + F4C0Ex^13 + E8B38x^12 + 5EE89x^11 + F8BFFx^10 + 30304x^9 + 5B9D4x^8 + 12653x^7 + 4DFB5x^6 + 879F8x^5 + 825FBx^4 + F1FFEx^3 + 3FFFFx^2 + 60000x^1 + 10000
